In [1]:
import torch
import torchvision
from torchvision import models,transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torch import optim

# Loading VGG pretrained model

In [2]:
vgg=models.vgg19(pretrained=True).features
vgg

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

Convert image to Tensor, resize it and normalize

In [3]:
import cv2
def load_image(img,msize):
    im=cv2.imread(img)
    if(np.max(im.size)>msize):
        shape=msize
    else:
        shape=max(im.size)
        
    transform=transforms.Compose([transforms.ToTensor(),
                                  transforms.Resize(shape),
                                  transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))
                                 ])
    im=transform(im).unsqueeze(0)
    return im

Load content and style images and trainsform then

In [4]:
style=load_image('impresionism.jpg',300)
content=load_image('bg.jpg',300)


# Show image

Convert image from Tensor to numpy and denormalize it

In [5]:
def prepare_img(im):
    im=im.detach().numpy().squeeze(0)
    im=im.transpose(1,2,0)
    im=im*np.array((0.485, 0.456, 0.406))+np.array((0.229, 0.224, 0.225))
    return im
    

In [6]:
im_show=prepare_img(style)


# Calculate features of convolutional layers for content and style image

Freeze parameters of pretrained VGG16

In [7]:
for param in vgg.parameters():
    param.requires_grad=False

Calculate features

In [8]:
def calc_features(im,model):
    
    features={}
    
    layers={'0':'conv_1_1',
            '5':'conv_2_1',
            '10':'conv_3_1',
            '19':'conv_4_1',
            '21':'conv_4_2',
            '28':'conv_5_1'
           }
    
    x=im
    
    for name,lay in model._modules.items():
        x=lay(x)
        if(name in layers):
            features[layers[name]]=x
            
    return features
        

# Calculate gram matrix as correlation matrix of features of every layer in features variable in style image

Calculate features of convolutional layers for content and style image

In [9]:
content_features=calc_features(content,vgg)
style_features=calc_features(style,vgg)

In [10]:
def gram_matrix(ft):
    
    matrix=ft.view(ft.shape[1],ft.shape[2]*ft.shape[3])
    gram=torch.mm(matrix,matrix.t())
        
    return gram

In [11]:
style_gram={}

for name in style_features:
    ft=style_features[name]
    style_gram[name]=gram_matrix(ft)
    


Create target image which is in first step content image and its parameters are updating during training according to total loss which is $$total loss=\alpha *content loss+ \beta*style loss$$
Our target image needs to have features of 2nd layer from 4th convolutional stack of VGG-19 like features of content image. 
Style of the target image needs to be similar as style of style image, so gram matrix of choosen layers needs of the target image needs to be similar to gram matrix of choosen layers of style image.
$$content loss=\frac{1}{2}\sum(T_c-C_c)^2$$,
where $T_c$ are features from target image from selected layers and $C_c$ are features from content image from selected layers.
$$style loss=a\sum{_i} w_i(T_s,i-S_s,i)^2$$,
where $T_s,i$ is gram matrix of target image of i-th feature and $S_s,i$ is gram matrix of style image of i-th feature.


In [12]:
target=content.clone().requires_grad_(True)
target.shape

torch.Size([1, 3, 300, 535])

Setting up weights for layers in calculating style loss. Range for weight is from 0 to 1 and deeper layers have lower weight than first layers.
Setting up content weight and style weight for calculating total loss as
$$total loss=$$

In [13]:
content_w=1
style_w=10^6
style_weights={'conv_1_1':1,
                 'conv_2_1':0.75,
                 'conv_3_1':0.5,
                 'conv_4_1':0.2,
                 'conv_4_2':0.2,
                 'conv_5_1':0.2
                }

Training style transfer 

In [ ]:
optimizer=optim.Adam([target],lr=0.005)
n_epochs=10000
n=200
for i in range(n_epochs):
    target_features=calc_features(target,vgg)
    cont_loss=torch.mean((content_features['conv_4_2']-target_features['conv_4_2'])**2)
    style_loss=0
    for name in style_features:
        target_gram=gram_matrix(target_features[name])
        sl=style_weights[name]*torch.mean((target_gram-style_gram[name])**2)
        style_loss+=sl/(target.shape[1]*target.shape[2]*target.shape[3])
        
    total_loss=style_w*style_loss+content_w*cont_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    
    if((i+1)%n==0):
        plt.imsave('style_transfer/image_new'+str(i)+'.jpg',prepare_img(target).clip(0,1))
        print(i,"Total loss",total_loss.item())

C:\Users\minap\anaconda3\lib\site-packages\torch\autograd\__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


199 Total loss 878.6083374023438
399 Total loss 379.2178039550781
599 Total loss 226.96383666992188
799 Total loss 168.22845458984375
999 Total loss 141.6485137939453
1199 Total loss 127.11407470703125
1399 Total loss 117.89907836914062
1599 Total loss 111.49398040771484
1799 Total loss 106.68290710449219
1999 Total loss 102.84024810791016
2199 Total loss 99.65975952148438
2399 Total loss 96.97737121582031
2599 Total loss 94.66635131835938
2799 Total loss 92.6919174194336
2999 Total loss 91.00465393066406
3199 Total loss 89.55375671386719
3399 Total loss 88.300537109375
3599 Total loss 87.23583984375
3799 Total loss 86.2478256225586
3999 Total loss 85.41835021972656
4199 Total loss 84.66259002685547
4399 Total loss 84.08453369140625
4599 Total loss 83.47266387939453
4799 Total loss 82.93267822265625
4999 Total loss 82.5615463256836
5199 Total loss 82.1283187866211
5399 Total loss 81.76302337646484
5599 Total loss 81.35369110107422
5799 Total loss 81.05326843261719
5999 Total loss 80.79